## Leitura de corrida realizada

In [ ]:
#Lê todos os dados que existem para uma entidade, e junta todos os atributos em um dataframe. Preciso modificar a entidade na mão

import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import json

    
def fetch_paginated_data(url_base, attribute_name, h_limit=100, is_location=False):
    """
    Busca dados usando paginação com hLimit e hOffset.
    """
    dict_data = {attribute_name: [], 'recvTime_' + attribute_name: []}
    h_offset = 0

    while True:
        # Construir URL com hLimit e hOffset
        url = f"{url_base}?hLimit={h_limit}&hOffset={h_offset}"
        headers = {
            'fiware-service': 'smart',
            'fiware-servicepath': '/'
        }
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            values = data['contextResponses'][0]['contextElement']['attributes'][0]['values']
            if not values:
                break  # Interrompe se não houver mais valores

            for value in values:
                dict_data[attribute_name].append(
                    value['attrValue']['coordinates'] if is_location else value['attrValue']
                )
                dict_data['recvTime_' + attribute_name].append(value['recvTime'])

            # Incrementa o offset para a próxima página
            h_offset += h_limit
        else:
            print(f"Erro na requisição: {response.status_code}")
            break

    return pd.DataFrame(dict_data)


# Definir atributos e URLs
attributes = [
    ("velocidade", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/velocidade"),
    ("rpm", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/rpm"),
    ("temperatura", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/temperature"),
    ("pressao", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/pressure"),
    ("dataColetaDados", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/dataColetaDados"),
    ("location", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/location"),
    ("engineload", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/engineload"),
    ("idCorrida", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/idCorrida"),
    ("throttlePosition", "http://20.62.95.153:8666/STH/v1/contextEntities/type/iot/id/urn:ngsi-ld:grandSiena:ewl6143/attributes/throttlePosition"),
]

# DataFrames para armazenar os resultados
dfs = {attribute: pd.DataFrame() for attribute, _ in attributes}

# Requisições paralelas para todos os atributos
with ThreadPoolExecutor() as executor:
    future_to_attribute = {
        executor.submit(fetch_paginated_data, url, attribute, 100, attribute == "location"): attribute
        for attribute, url in attributes
    }
    for future in future_to_attribute:
        attribute = future_to_attribute[future]
        try:
            result = future.result()
            dfs[attribute] = result
        except Exception as e:
            print(f"Erro ao buscar {attribute}: {e}")

# Combinar os DataFrames baseando-se nos timestamps
df = dfs["velocidade"]
for attribute in [attr for attr, _ in attributes if attr != "velocidade"]:
    df = df.merge(
        dfs[attribute],
        how="outer",
        left_on="recvTime_velocidade",
        right_on=f"recvTime_{attribute}"
    )

# Limpar o DataFrame final

df = df.drop(columns=['recvTime_velocidade', 'recvTime_rpm', 'recvTime_temperatura', 'recvTime_pressao', 'recvTime_dataColetaDados',  'recvTime_location', 'recvTime_engineload', 'recvTime_idCorrida', 'recvTime_throttlePosition'])
df = df.dropna()

# Exibir o resultado
df_raw = df.copy()

df_raw



## Tratamento de dados 

A partir do dataset coletado, é preciso fazer algumas correções. 

- Converter as colunas para cada tipo
- Retirar casos inconsistentes. Exemplo: data = 0, location[0,0], dados com -999
- Encontrar as corridas que existem nas entidades
- Separar o location em lat e log

In [ ]:
import datetime

df = df_raw

#Conversão do tipo de dado.
columns = {
    'idCorrida': int,
    'velocidade': int,
    'rpm': float,
    'temperatura': int,
    'pressao': int,
    'engineload': float,
    'throttlePosition': float,
    'dataColetaDados': datetime,
    'location': str,
}

#Retirar casos em que o dia veio como 0. É uma inconsistencia geral
df = df[df['dataColetaDados'] != '0']


for column, dtype in columns.items():
    if dtype == datetime:
        df[column] = pd.to_datetime(df[column])
    else:
        df[column] = df[column].astype(dtype)

df = df.set_index('dataColetaDados')

# Remover valores nulos ou inconsistentes em 'location'
df = df[df['location'].notna() & df['location'].str.startswith('[')]

# Separar latitude e longitude usando split
df['latitude'] = df['location'].apply(lambda x: x.strip('[]').split(', ')[0]).astype(float)
df['longitude'] = df['location'].apply(lambda x: x.strip('[]').split(', ')[1]).astype(float)

#Inconsistência de longitude e latitude serem 0
df = df[(df['latitude'] != 0) & (df['longitude'] != 0)]

#Removendo valores extremos. Normalmente eles valem -999
df = df[(df['velocidade'] != -999) & (df['rpm'] != -999) & (df['pressao'] != -999) & (df['temperatura'] != -999) & (df['engineload'] != -999) & (df['throttlePosition'] != -999)]

#Considero um unico dataframe, pois o processo é mais linear
df_processado = df[['velocidade', 'rpm', 'temperatura', 'pressao', 'engineload', 'throttlePosition', 'location', 'latitude', 'longitude', 'idCorrida']].copy()


## Tratamento de Dados - Correção de coordenada
- Correção de latitude e longitude usando API google Snap to Roads.

In [ ]:
# Transformar o índice em uma coluna e resetar o índice
df_processado = df_processado.reset_index()

# Renomear a coluna do índice, se necessário
df_processado = df_processado.rename(columns={'index': 'novo_indice'})

# Chave para acesso a API do Google Maps. Substitua pela sua chave
api_key = 

# Número máximo de coordenadas por batch
batch_size = 100
new_coords = {}
new_place_ids = {}

# Iterar sobre o DataFrame em batches
for start in range(0, len(df_processado), batch_size):
    end = start + batch_size
    batch = df_processado.iloc[start:end]

    # Criar o path para a requisição
    path = '|'.join([f"{lat},{lon}" for lat, lon in zip(batch['latitude'], batch['longitude'])])

    # Fazer a requisição para o Google Maps API
    url = f"https://roads.googleapis.com/v1/snapToRoads?path={path}&key={api_key}"
    response = requests.get(url)
    snapped_points = response.json()

    # Extrair as novas coordenadas e placeIds, ajustando o índice para o DataFrame original
    for point in snapped_points.get('snappedPoints', []):
        relative_index = point['originalIndex']  # Índice dentro do batch (0-99)
        original_index = relative_index + start  # Índice absoluto no DataFrame original
        # Adicionar nova coordenada
        new_coords[original_index] = (
            point['location']['latitude'], 
            point['location']['longitude']
        )
        # Adicionar placeId, se disponível
        if 'placeId' in point:
            new_place_ids[original_index] = point['placeId']

# Adicionar as novas coordenadas ao DataFrame original
df_processado['new_latitude'] = df_processado.index.map(lambda idx: new_coords[idx][0] if idx in new_coords else None)
df_processado['new_longitude'] = df_processado.index.map(lambda idx: new_coords[idx][1] if idx in new_coords else None)

# Adicionar a coluna de placeId ao DataFrame original
df_processado['placeId'] = df_processado.index.map(lambda idx: new_place_ids[idx] if idx in new_place_ids else None)

# Exibir o DataFrame atualizado
df_processado

## Critérios - Limite de Velocidade

- Por conta do baixo limite de requisições gratuitas da API da Azure, é preciso reduzir as requisições. Os 3 blocos abaixo avaliam limites de velocidade já coletados, e só requisitão os novos casos

In [ ]:
#Linhas que não receberam novas coordenadas são removidas
df_processado = df_processado.dropna(subset=['new_latitude', 'new_longitude'])

#Parquet que armazena placeId e o limite de velocidade
df_limites_cache = pd.read_parquet("limites_velocidade.parquet")

#Verificando se existem novos limites de velocidade a ser capturado
df_limites_faltantes = df_processado.merge(df_limites_cache, on='placeId', how='left')
df_limites_faltantes = df_limites_faltantes[df_limites_faltantes['limite_velocidade'].isna()]

if(len(df_limites_faltantes) > 0):
    
    #Criar um novo parquet com os novos limites
    df_limites_faltantes = df_limites_faltantes.drop_duplicates(subset=['placeId'], ignore_index=True)
    print(len(df_limites_faltantes))
    df_limites_faltantes = df_limites_faltantes.drop(columns=['limite_velocidade', 'Street'])


- Utilizando API Azure para encontrar limite da via ( Somente os casos que ainda não tem limite)

In [ ]:
if(len(df_limites_faltantes) > 0):

    import json

    # Chave para acesso a API do Azure. Substitua pela sua chave
    key = 
    
    # Definindo o URL do endpoint batch
    url = f"https://atlas.microsoft.com/search/address/reverse/batch/json?api-version=1.0&subscription-key={key}"

    # Criando o payload dinamicamente a partir do DataFrame. Não precisa ter limite de 100 coordenadas para a API da Azure
    batch_items = []

    #Precisa resetar o index mais uma vez
    # Transformar o índice em uma coluna e resetar o índice
    #df_limites_faltantes = df_limites_faltantes.reset_index()

    # Renomear a coluna do índice, se necessário
    #df_limites_faltantes = df_limites_faltantes.rename(columns={'index': 'novo_indice'})

    for _, row in df_limites_faltantes.iterrows():
        latitude = row['new_latitude']
        longitude = row['new_longitude']
        # Adiciona um item ao batch usando o formato específico da query
        batch_items.append({
            "query": f"?query={latitude},{longitude}&returnSpeedLimit=true"
        })

    # Monta o payload final
    payload = {
        "batchItems": batch_items
    }

    headers = {
        'Content-Type': 'application/json'  # Importante: definir corretamente o Content-Type
    }


    # Fazendo a requisição para criar o job
    response = requests.post(url, data=json.dumps(payload), headers=headers)

    # Verificando o resultado da criação do job
    if response.status_code == 202:
        # O job foi aceito e a URL para verificar o status foi retornada
        operation_location = response.headers['location']
        print(f"Job criado com sucesso! Verifique o status em: {operation_location}")

        result_response = requests.get(operation_location)
        result_data = result_response.json()

        # Lista para armazenar os dados extraídos
        data = []

        # Extraindo os dados do JSON
        for item in result_data['batchItems']:
            address_info = item.get('response', {}).get('addresses', [{}])[0].get('address', {})
            street_name = address_info.get('street', 'Desconhecido')
            speed_limit = address_info.get('speedLimit', 'Sem velocidade')
            data.append({'Street': street_name, 'Speed Limit': speed_limit})

        # Criando um DataFrame a partir dos dados extraídos
        df_limites = pd.DataFrame(data)

        # Combinando os DataFrames
        df_limites_faltantes = df_limites_faltantes.merge(df_limites, left_index=True, right_index=True)

    else:
        print(f"Erro ao criar o job: {response.status_code} - {response.text}")

- Depois de encontrar os limites faltantes, precisa salvar no parquet.
- Observação: podem haver casos que não encontrou um limite para o placeId. Mesmo nesse caso, o placeId é salvo com limite 0, para evitar redundancia e um gasto maior da API

In [ ]:
if(len(df_limites_faltantes) > 0):
    #Captura somente o valor numerico do limite de velocidade e converte para float
    df_limites_faltantes['limite_velocidade'] = df_limites_faltantes['Speed Limit'].str.extract(r'(\d+)').astype(float)

    #Adicionar limite de velocidade como 0 nos casos em que não foi possível encontrar
    df_limites_faltantes.loc[df_limites_faltantes['limite_velocidade'].isna(), 'limite_velocidade'] = 0

    #Passar os novos limites para o parquet - Os placeId serao unicos???
    df_limites_cache = pd.concat([df_limites_cache, df_limites_faltantes[['placeId', 'Street','limite_velocidade']]], ignore_index=True)
    df_limites_cache.to_parquet('limites_velocidade.parquet')

#Adicionar coluna de limite de velocidade
df_processado = df_processado.merge(df_limites_cache, on='placeId', how='left')

#Retira da analise os casos em que a velocidade é 0 ou o limite de velocidade é 0. Pois podem representar tanto uma situação de um veiculo parado, como uma inconsistencia na base de dados.
#df_processado = df_processado[(df_processado['velocidade'] > 0) & (df_processado['limite_velocidade'] != 0)]

    

Aceleração e Desaceleração

In [ ]:
df_processado = df_processado.set_index('dataColetaDados')

#Precisa converter a velocidade para m/s, pois o limite de velocidade vem em km/h
df_processado['velocidade_convertida'] = df_processado['velocidade'] / 3.6
df_processado['diff_tempo'] = df_processado.index.to_series().diff().dt.total_seconds()
df_processado['diff_velocidade'] = df_processado['velocidade_convertida'].diff()  

df_processado['aceleracao_derivada'] = df_processado['diff_velocidade'] / df_processado['diff_tempo']

#Inconsistencia de corridas que os dados deram muitos alto, ou muito baixo, uma situação irreal
df_processado = df_processado[(df_processado['aceleracao_derivada'] > -10) & (df_processado['aceleracao_derivada'] < 10)]

## Classificação

In [ ]:
import joblib
from sklearn.preprocessing import StandardScaler


df_processado_agressivo = df_processado[df_processado['limite_velocidade'] != 0]

# Seleciona as colunas usadas no treinamento
feature_cols = ['velocidade', 'rpm', 'temperatura', 'pressao', 
                'engineload', 'throttlePosition', 'limite_velocidade', 'aceleracao_derivada']

# Prever a classificação para o dataset "agressivo"
df_agressivo_ml = df_processado_agressivo.copy()
X_agressivo = df_agressivo_ml[feature_cols]
scaler = StandardScaler()
scaler.fit(X_agressivo)
X_agressivo_scaled = scaler.transform(X_agressivo)

# Prever a classificação usando o modelo SVM
svm_model = joblib.load("svm_model.joblib")
df_agressivo_ml['classificacao'] = svm_model.predict(X_agressivo_scaled)

# Combina os datasets classificados (mantendo as coordenadas) para uso no mapa
df_geracao = df_agressivo_ml

In [ ]:
import folium

# Cria o mapa centralizado na primeira coordenada
m = folium.Map(location=[df_geracao['new_latitude'].iloc[0], 
                         df_geracao['new_longitude'].iloc[0]], 
               zoom_start=15)

# Adiciona os pontos e linhas ao mapa
for i in range(len(df_geracao) - 1):
    # Coordenadas do ponto atual e do próximo ponto
    lat1, lon1 = df_geracao.iloc[i][['new_latitude', 'new_longitude']]
    lat2, lon2 = df_geracao.iloc[i + 1][['new_latitude', 'new_longitude']]
    
    # Classificação do ponto atual
    classificacao = df_geracao.iloc[i]['classificacao']
    
    # Define a cor da linha com base na classificação
    line_color = 'red' if classificacao == 0 else 'green'
    
    # Adiciona uma linha entre o ponto atual e o próximo
    folium.PolyLine([(lat1, lon1), (lat2, lon2)], color=line_color, weight=5).add_to(m)
    
    # Adiciona um marcador no ponto atual
    #folium.Marker(
    #    location=[lat1, lon1],
    #    popup=f"Coordenadas: ({lat1}, {lon1})<br>Classificação: {classificacao}",
    #   icon=folium.Icon(color='blue', icon='info-sign')
    #).add_to(m)

# Salva o mapa em um arquivo HTML e exibe o mapa
output_file = r"C:\Users\vidal\Downloads\TCC\ClassificacaoCondutores\percurso_final.html"
m.save(output_file)
output_file